# NLP_GoingDeeper | P07.BERT
---


1. Tokenizer 준비
2. 데이터 전처리 (1) MASK 생성
3. 데이터 전처리 (2) NSP pair 생성
4. 데이터 전처리 (3) 데이터셋 완성
5. BERT 모델 구현
6. pretrain 진행

In [1]:
! mkdir -p ~/aiffel/bert_pretrain/data
! mkdir -p ~/aiffel/bert_pretrain/models
! ln -s ~/data/kowiki.txt ~/aiffel/bert_pretrain/data/kowiki.txt

ln: failed to create symbolic link '/aiffel/aiffel/bert_pretrain/data/kowiki.txt': File exists


In [2]:
# imports
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import tensorflow.keras.backend as K

import os
import re
import math
import numpy as np
import pandas as pd
import random
import collections
import json
import shutil
import zipfile
import copy
from datetime import datetime

import matplotlib.pyplot as plt
import sentencepiece as spm
from tqdm.notebook import tqdm

random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

# tf version 및 gpu 확인
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print(tf.test.gpu_device_name())

2.4.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


- BERT에 사용되는 [MASK], [SEP], [CLS] 등의 주요 특수문자가 vocab에 포함되어야 함

In [3]:
# import sentencepiece as spm
# import os
# corpus_file = os.getenv('HOME')+'/aiffel/bert_pretrain/data/kowiki.txt'
# prefix = 'ko_32000'
# vocab_size = 32000

# spm.SentencePieceTrainer.train(
#     f"--input={corpus_file} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
#     " --model_type=bpe" +
#     " --max_sentence_length=999999" + # 문장 최대 길이
#     " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
#     " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
#     " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
#     " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
#     " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰


In [4]:
# ! mv ko_32000.* ~/aiffel/bert_pretrain/models

In [5]:
! ln -s ~/data/ko_32000.model ~/aiffel/bert_pretrain/models/ko_32000.model
! ln -s ~/data/ko_32000.vocab ~/aiffel/bert_pretrain/models/ko_32000.vocab


ln: failed to create symbolic link '/aiffel/aiffel/bert_pretrain/models/ko_32000.model': File exists
ln: failed to create symbolic link '/aiffel/aiffel/bert_pretrain/models/ko_32000.vocab': File exists


In [ ]:
data_dir = os.getenv('HOME')+'/aiffel/bert_pretrain/data'
model_dir = os.getenv('HOME')+'/aiffel/bert_pretrain/models'

# vocab loading
vocab = spm.SentencePieceProcessor()
vocab.load(f"{model_dir}/ko_32000.model")

In [ ]:
# 특수 token 7개를 제외한 나머지 tokens 들
vocab_list = []
for id in range(7, len(vocab)):
    if not vocab.is_unknown(id):
        vocab_list.append(vocab.id_to_piece(id))
print(vocab_list)

In [ ]:
# [CLS], tokens a, [SEP], tokens b, [SEP] 형태의 token 생성
string_a = "추적추적 비가 내리는 날이었어 그날은 왠지 손님이 많아 첫 번에 삼십 전 둘째번 오십 전 오랜만에 받아보는 십 전짜리 백통화 서푼에"
string_b = "손바닥 위엔 기쁨의 눈물이 흘러 컬컬한 목에 모주 한잔을 적셔 몇 달 포 전부터 콜록거리는 아내 생각에 그토록 먹고 싶다던"
tokens_org = ["[CLS]"] + vocab.encode_as_pieces(string_a) + ["[SEP]"] + vocab.encode_as_pieces(string_b) + ["[SEP]"]
print(tokens_org)

In [ ]:
print(tokens_org)

# 전체 token의 15% mask
mask_cnt = int((len(tokens_org) - 3) * 0.15)
mask_cnt

In [ ]:
# random mask를 위해서 순서를 섞음
random.shuffle(cand_idx)
cand_idx

In [ ]:
# random mask를 위해서 순서를 섞음
random.shuffle(cand_idx)
cand_idx

In [ ]:
# tokens가 mask되므로 재 실행을 위해서 넣어줌 (테스트용)
tokens = copy.deepcopy(tokens_org)

mask_lms = []  # mask 된 값
for index_set in cand_idx:
    if len(mask_lms) >= mask_cnt:  # 핸재 mask된 개수가 15%를 넘으면 중지
          break
    if len(mask_lms) + len(index_set) > mask_cnt:  # 이번에 mask할 개수를 포함해 15%를 넘으면 skip
          continue
    dice = random.random()  # 0..1 사이의 확률 값

    for index in index_set:
        masked_token = None
        if dice < 0.8:  # 80% replace with [MASK]
            masked_token = "[MASK]"
        elif dice < 0.9: # 10% keep original
            masked_token = tokens[index]
        else:  # 10% random word
            masked_token = random.choice(vocab_list)
        mask_lms.append({"index": index, "label": tokens[index]})
        tokens[index] = masked_token

print("tokens_org")
print(tokens_org, "\n")
print("tokens")
print(tokens)

In [ ]:
# 순서 정렬 및 mask_idx, mask_label 생성
mask_lms = sorted(mask_lms, key=lambda x: x["index"])
mask_idx = [p["index"] for p in mask_lms]
mask_label = [p["label"] for p in mask_lms]

print("mask_idx   :", mask_idx)
print("mask_label :", mask_label)